In [5]:
#! pip install gmaps

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [8]:
#Load the csv exported in Part I to a DataFrame

weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.37,-73.41,70.00,94,1,3.36,US,1595650678
1,Inza,53.85,46.36,60.06,69,28,3.74,RU,1595650925
2,Barrow,71.29,-156.79,35.60,80,90,9.17,US,1595650925
3,George Town,5.41,100.34,87.80,74,20,2.24,MY,1595650878
4,Vaini,-21.20,-175.20,71.60,88,75,8.05,TO,1595650925
...,...,...,...,...,...,...,...,...,...
582,Anshun,26.25,105.93,75.49,85,100,14.20,CN,1595651024
583,Tual,-5.67,132.75,79.70,78,77,20.80,ID,1595651024
584,Giohar,2.78,45.50,75.76,71,60,17.22,SO,1595651024
585,Narón,43.52,-8.15,64.00,79,2,1.99,ES,1595651025


In [9]:
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Lat", "Lng"]]
#Add Heatmap layer to map.
humidity = weather_data["Humidity"]

In [11]:
# map for locations and humidity
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
narrowed_city = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Jalu,29.03,21.55,73.80,45,0,7.90,LY,1595650931
39,Albany,42.60,-73.97,72.00,73,0,2.37,US,1595650932
52,Ribeira Grande,38.52,-28.70,70.00,79,0,8.79,PT,1595650648
183,Takoradze,4.88,-1.76,73.92,91,0,9.40,GH,1595650958
192,Carnarvon,-24.87,113.63,78.80,22,0,9.17,AU,1595650960
213,Coxim,-18.51,-54.76,71.02,43,0,3.89,BR,1595650964
215,Ubari,26.59,12.78,73.31,26,0,8.93,LY,1595650964
260,Campo Grande,-20.44,-54.65,71.60,53,0,5.82,BR,1595650971
272,Nador,35.17,-2.93,77.00,38,0,4.70,MA,1595650973
294,Esim,4.87,-2.24,74.10,91,0,9.31,GH,1595650977


In [14]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
34,Jalu,LY,29.03,21.55,
39,Albany,US,42.60,-73.97,
52,Ribeira Grande,PT,38.52,-28.70,
183,Takoradze,GH,4.88,-1.76,
192,Carnarvon,AU,-24.87,113.63,
213,Coxim,BR,-18.51,-54.76,
215,Ubari,LY,26.59,12.78,
260,Campo Grande,BR,-20.44,-54.65,
272,Nador,MA,35.17,-2.93,
294,Esim,GH,4.87,-2.24,


In [17]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [19]:
hotel_df    

,City,Country,Lat,Lng,Hotel Name
34,Jalu,LY,29.03,21.55,
39,Albany,US,42.60,-73.97,
52,Ribeira Grande,PT,38.52,-28.70,
183,Takoradze,GH,4.88,-1.76,
192,Carnarvon,AU,-24.87,113.63,
213,Coxim,BR,-18.51,-54.76,
215,Ubari,LY,26.59,12.78,
260,Campo Grande,BR,-20.44,-54.65,
272,Nador,MA,35.17,-2.93,
294,Esim,GH,4.87,-2.24,


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))